# Learning Apache Spark with PySpark
*An introduction to basic concepts for working with Apache Spark using Python.*

*This notebook implements these concepts in code, serving both as a learning resource and as a collection of reusable code snippets for future projects.*


## What is Apache Spark?

Apache Spark is a tool designed to process and analyze large amounts of data efficiently.
Instead of working on data row by row on a single machine, Spark is built to split work into smaller pieces and process them in parallel.

Although Spark is often used on clusters with many machines, it can also run locally on a single computer.
In this notebook, Spark is used in **local mode**, which makes it easier to experiment and learn, while still using the same APIs that would later scale to a cluster.

At a high level, Spark focuses on:
- processing data in parallel,
- keeping data in memory when possible for better performance,
- delaying execution until a result is actually needed.

These ideas make Spark especially useful in Big Data and Cloud Computing contexts, where datasets are too large to be handled efficiently by traditional single-machine tools.


## SparkSession: Entry point to Spark
A SparkSession is the main entry point to Apache Spark when using it from Python (PySpark).

In practical terms, the SparkSession:
connects your Python code to the Spark engine (running on the JVM), manages configuration and resources, allows you to create DataFrames, read data, and run computations.

Without a SparkSession, Spark has no context in which to execute your code.

You need to import the SparkSession from PySpark in order to start writing any Spark application, then initialize a session:

In [1]:
# Import the SparkSession
from pyspark.sql import SparkSession

# Start the SparkSession,
spark = (SparkSession
         .builder
         .appName("Test-1")
         .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/26 14:59:38 WARN Utils: Your hostname, MacBook-Air-de-Martin.local, resolves to a loopback address: 127.0.0.1; using 192.168.4.46 instead (on interface en0)
26/01/26 14:59:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/26 14:59:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Two Ways of Working with Data in Spark: DataFrames and RDDs

Apache Spark offers two main APIs for working with distributed data:

- **DataFrames**: high-level, optimized, schema-based (used in practice)
- **RDDs (Resilient Distributed Datasets)**: low-level, flexible, conceptual

In this notebook, we primarily use **DataFrames**, since they are the modern and recommended approach.
However, understanding some RDD concepts helps explain *why* certain DataFrame operations behave the way they do.

A useful mental model when working with Spark is to always ask:

*“What does **one element** represent at this point in the pipeline?"*

This question applies to both DataFrames and RDDs.



## DataFrames
A **Spark DataFrame** is Spark’s main way of working with structured data.  
It looks and feels similar to a Pandas DataFrame, but it is designed to work with much larger datasets.

Behind the scenes, a Spark DataFrame:
- is split into multiple partitions,
- is evaluated lazily (nothing runs until a result is needed),
- is optimized automatically by Spark before execution.

Because of this, DataFrames are what you’ll use most of the time when working with Spark in PySpark and Spark SQL.

### Defining Data

To create a DataFrame, two components are required:

Data – the actual rows
Schema – the structure of the DataFrame (column names and types)

This components can be declared in the script, or read from different data files.

**Declaring the data manually:**

In [2]:
# Declaring the different entities (rows) of the DataFrame
data = [[1, "Nombre 1", "Apellido 1"],
        [2, "Nombre 2", "Apellido 2"],
        [3, "Nombre 3", "Apellido 3"]]

**Creating the Schema:**
It can be done in different ways, here are two simple ones.

In [3]:
# Create the schema for the DataFrame, using SQL DDL
schema = "`ID` INT, `First` STRING, `Second` STRING"

# Create the "schema" by naming the columns in another python list
columns = ["ID", "Fname", "Lname"]

### Creating Spark DataFrames

Spark DataFrames are created using the active SparkSession.
The same data can produce different DataFrames depending on how the schema is provided.

In [4]:
# From the schema
my_first_df = spark.createDataFrame(data, schema)

# From the columns list
my_second_df = spark.createDataFrame(data, columns)

### Displaying the DataFrame

In [5]:
# Printing the DataFrames
my_first_df.show()
my_second_df.show()

# Method to print the DataFrames schema structures
print(my_first_df.printSchema())
print(my_second_df.printSchema())

+---+--------+----------+
| ID|   First|    Second|
+---+--------+----------+
|  1|Nombre 1|Apellido 1|
|  2|Nombre 2|Apellido 2|
|  3|Nombre 3|Apellido 3|
+---+--------+----------+

+---+--------+----------+
| ID|   Fname|     Lname|
+---+--------+----------+
|  1|Nombre 1|Apellido 1|
|  2|Nombre 2|Apellido 2|
|  3|Nombre 3|Apellido 3|
+---+--------+----------+

root
 |-- ID: integer (nullable = true)
 |-- First: string (nullable = true)
 |-- Second: string (nullable = true)

None
root
 |-- ID: long (nullable = true)
 |-- Fname: string (nullable = true)
 |-- Lname: string (nullable = true)

None


## RDDs: Conceptual Foundation

Although this notebook primarily uses **DataFrames**, it is useful to briefly understand
**RDDs (Resilient Distributed Datasets)**, as they form the conceptual foundation of Spark.

RDDs are managed by the **SparkContext**, which is already available through the
`SparkSession`. This is why RDDs are created via:


```spark.sparkContext```

An RDD cannot be created directly like a Python list, because it represents a
distributed collection that requires Spark’s execution engine, scheduler, and lineage tracking.


### Creating an RDD

In [ ]:
rdd = spark.sparkContext.parallelize([
    "This is a line",
    "Another line"
])
# At this point, each element of the RDD is one line (a string).


### `map` vs `flatMap`

In [ ]:
# Using map
rdd_map = rdd.map(lambda line: line.split())
rdd_map.collect()

[['This', 'is', 'a', 'line'], ['Another', 'line']]

Here:

- each input element (a line) produces one output element

- the output elements are lists of words

- So the RDD contains lists, not individual words.

---

In [10]:
# Using flatMap
rdd_flatmap = rdd.flatMap(lambda line: line.split())
rdd_flatmap.collect()

['This', 'is', 'a', 'line', 'Another', 'line']

Here:

- one input element produces multiple output elements

- the lists are flattened

- each RDD element is now one word
---

## Text Processing with DataFrames

In this section, we read and process a text file using **DataFrames**.
Although DataFrames are used for the implementation, the operations closely
relate to the RDD concepts introduced earlier.

---

### Reading a text file:

In [13]:
df = spark.read.text("text_example.txt")
df.show(truncate=False)

+------------------------------------------------------------------------+
|value                                                                   |
+------------------------------------------------------------------------+
|Apache Spark is a powerful engine for large-scale data processing.      |
|Spark allows data to be processed in parallel across multiple machines. |
|This notebook explores how Spark handles text data using DataFrames.    |
|Understanding how data is transformed helps avoid common mistakes.      |
|Spark provides both low-level and high-level APIs for working with data.|
+------------------------------------------------------------------------+



When reading a text file with spark.read.text:

- Each row represents one line of the file
- In this case, the DataFrame contains a single column, usually named value

This is analogous to an RDD where each element corresponds to one line.

At this stage, the mental model is:

>*one row = one line of text*

---

### Splitting lines into words:


In [14]:
from pyspark.sql.functions import split, explode

words_df = df.select(
    explode(split(df.value, " ")).alias("word")
)

words_df.show()


+-----------+
|       word|
+-----------+
|     Apache|
|      Spark|
|         is|
|          a|
|   powerful|
|     engine|
|        for|
|large-scale|
|       data|
|processing.|
|      Spark|
|     allows|
|       data|
|         to|
|         be|
|  processed|
|         in|
|   parallel|
|     across|
|   multiple|
+-----------+
only showing top 20 rows


Here, two operations are combined:

- `split` transforms each line into a list of words

- `explode` flattens these lists so that each word becomes its own row

Conceptually:

- `split` produces a structure similar to `map(lambda line: line.split())`

- `explode` plays a role similar to `flatMap`, flattening the result

After this step, the mental model becomes:

>*one row = one word*

---

### Counting words:

In [18]:
word_counts = words_df.groupBy("word").count()
word_counts.show()


+-----------+-----+
|       word|count|
+-----------+-----+
|  mistakes.|    1|
|    handles|    1|
|      using|    1|
|        for|    2|
|        how|    2|
|   provides|    1|
|   powerful|    1|
|      data.|    1|
|         in|    1|
|       with|    1|
|         be|    1|
|  processed|    1|
|  machines.|    1|
|       both|    1|
|     Apache|    1|
|     allows|    1|
|         is|    2|
|   parallel|    1|
|DataFrames.|    1|
|       data|    4|
+-----------+-----+
only showing top 20 rows


This groups identical words and counts how many times each appears.

Because DataFrames are schema-based and optimized, Spark can:

- reorder operations

- reduce data movement

- execute the aggregation efficiently

This is one of the main advantages of using DataFrames over RDDs for
structured operations like counting and grouping.

---

**Important note on actions**

Operations such as `show()` trigger execution of the Spark pipeline.

Unlike RDDs:

- DataFrames provide `show()` for inspection

- RDDs use `collect()` to return data as a local Python list

In both cases, these operations should only be used on small datasets
for debugging or demonstration purposes.

---

## Text Processing with RDDs (MapReduce Mental Model)

To clearly contrast DataFrames with RDDs, we now perform the same text processing task
using RDDs. This helps illustrate the classic *MapReduce-style workflow* and makes the
differences between the two APIs explicit.

### Reading the text file as an RDD:

In [19]:
rdd = spark.sparkContext.textFile("text_example.txt")


At this point:

- each RDD element is one line of text

- this is conceptually equivalent to `spark.read.text(...)`

Before any transformations are applied:

> *one RDD element = one line*

---

### Splitting lines into words (flatMap)

In [20]:
words_rdd = rdd.flatMap(lambda line: line.split())
words_rdd.collect()

['Apache',
 'Spark',
 'is',
 'a',
 'powerful',
 'engine',
 'for',
 'large-scale',
 'data',
 'processing.',
 'Spark',
 'allows',
 'data',
 'to',
 'be',
 'processed',
 'in',
 'parallel',
 'across',
 'multiple',
 'machines.',
 'This',
 'notebook',
 'explores',
 'how',
 'Spark',
 'handles',
 'text',
 'data',
 'using',
 'DataFrames.',
 'Understanding',
 'how',
 'data',
 'is',
 'transformed',
 'helps',
 'avoid',
 'common',
 'mistakes.',
 'Spark',
 'provides',
 'both',
 'low-level',
 'and',
 'high-level',
 'APIs',
 'for',
 'working',
 'with',
 'data.']

Explanation:

- `line.split()` produces a list of words per line

- `flatMap` flattens these lists into individual elements

Mental model:

> *one RDD element = one word*

This corresponds directly to:

`split` + `explode` in the DataFrame 

---

### Mapping words to key–value pairs (map)

In [21]:
word_pairs = words_rdd.map(lambda word: (word, 1))
word_pairs.collect()

[('Apache', 1),
 ('Spark', 1),
 ('is', 1),
 ('a', 1),
 ('powerful', 1),
 ('engine', 1),
 ('for', 1),
 ('large-scale', 1),
 ('data', 1),
 ('processing.', 1),
 ('Spark', 1),
 ('allows', 1),
 ('data', 1),
 ('to', 1),
 ('be', 1),
 ('processed', 1),
 ('in', 1),
 ('parallel', 1),
 ('across', 1),
 ('multiple', 1),
 ('machines.', 1),
 ('This', 1),
 ('notebook', 1),
 ('explores', 1),
 ('how', 1),
 ('Spark', 1),
 ('handles', 1),
 ('text', 1),
 ('data', 1),
 ('using', 1),
 ('DataFrames.', 1),
 ('Understanding', 1),
 ('how', 1),
 ('data', 1),
 ('is', 1),
 ('transformed', 1),
 ('helps', 1),
 ('avoid', 1),
 ('common', 1),
 ('mistakes.', 1),
 ('Spark', 1),
 ('provides', 1),
 ('both', 1),
 ('low-level', 1),
 ('and', 1),
 ('high-level', 1),
 ('APIs', 1),
 ('for', 1),
 ('working', 1),
 ('with', 1),
 ('data.', 1)]

Here:

- each word is mapped to a tuple (word, 1)

- this prepares the data for aggregation

This step represents the ***Map*** phase in ***MapReduce***.

---

### Reducing by key (counting words)

In [22]:
word_counts_rdd = word_pairs.reduceByKey(lambda a, b: a + b)
word_counts_rdd.collect()

[('powerful', 1),
 ('for', 2),
 ('to', 1),
 ('parallel', 1),
 ('multiple', 1),
 ('machines.', 1),
 ('explores', 1),
 ('how', 2),
 ('handles', 1),
 ('text', 1),
 ('using', 1),
 ('Understanding', 1),
 ('transformed', 1),
 ('helps', 1),
 ('avoid', 1),
 ('low-level', 1),
 ('and', 1),
 ('working', 1),
 ('with', 1),
 ('data.', 1),
 ('Apache', 1),
 ('Spark', 4),
 ('is', 2),
 ('a', 1),
 ('engine', 1),
 ('large-scale', 1),
 ('data', 4),
 ('processing.', 1),
 ('allows', 1),
 ('be', 1),
 ('processed', 1),
 ('in', 1),
 ('across', 1),
 ('This', 1),
 ('notebook', 1),
 ('DataFrames.', 1),
 ('common', 1),
 ('mistakes.', 1),
 ('provides', 1),
 ('both', 1),
 ('high-level', 1),
 ('APIs', 1)]

Explanation:

- all values associated with the same key (word) are combined

- the values (1) are summed

This represents the ***Reduce*** phase in ***MapReduce***.

---


## Comparison with the DataFrame approach

When working with **RDDs**, the full logic of the computation is written out step by step:

- each transformation is explicit and low-level  
- we manually define how data is mapped and reduced  
- Spark executes the provided functions as **black boxes**, without knowing their intent  

This gives a lot of control, but also requires more code and makes optimization harder.

When working with **DataFrames**, the focus shifts from *how* to compute the result to *what* result we want:

- operations such as `groupBy` and `count` describe the intent declaratively  
- Spark can analyze the operations and choose an efficient execution plan  
- the resulting code is usually shorter and easier to read  

Both approaches produce the same final result.  
However, DataFrames allow Spark to apply automatic optimizations, which is why they are generally preferred in practice.
